此檔案為AIMed 0828的exam。


醫學系六年級 許育綸


本次model的形狀與組員林益帆同學一起討論。但模型建造及訓練都是各自完成。



# 版本處理

In [31]:
pip install  tensorflow==2.5.1

上面cell跑過就要restart一下

In [ ]:
import tensorflow as tf
print(tf.__version__)

# Q1. Residual Block Implementation

In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input, activations
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, ZeroPadding2D, MaxPooling2D, AveragePooling2D,Flatten, Dense 
from tensorflow.keras.utils import plot_model
from IPython.display import Image
import numpy as np
import os
from tensorflow.keras import regularizers as lr
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
input = keras.Input(shape=(128,128,64),name="first_input")
h1=layers.Conv2D(64,(1,1),)(input)
h2=layers.Conv2D(128,(3,3),padding='same',activation='relu')(h1)
h3=layers.Conv2D(64,(1,1),)(h2)
output=layers.add([input, h3])

model=keras.Model(inputs=input,outputs=output)
plot_model(model,to_file='first.png')
Image("first.png")

In [5]:
input = keras.Input(shape=(128,128,64),name="first_input")
h1=layers.Conv2D(64,(1,1),)(input)
h2_1=layers.Conv2D(128,(3,3),padding='same',activation='relu')(h1)
h2_2=layers.Conv2D(128,(3,3),padding='same',activation='relu')(h1)
h2_3=layers.Conv2D(128,(3,3),padding='same',activation='relu')(h1)
concat = layers.Concatenate()([h2_1,h2_2,h2_3])
h3=layers.Conv2D(64,(1,1),)(concat)
output=layers.add([input, h3])

model2=keras.Model(inputs=input,outputs=output)
plot_model(model2,to_file='second.png')
Image("second.png")


In [6]:
input = keras.Input(shape=(128,128,64),name="first_input")
h1=layers.Conv2D(64,(1,1),)(input)
h2_1=layers.Conv2D(128,(3,3),padding='same',activation='relu')(h1)
h2_2=layers.Conv2D(128,(3,3),padding='same',activation='relu')(h1)
h2_3=layers.Conv2D(128,(3,3),padding='same',activation='relu')(h1)
global_ave=layers.GlobalAveragePooling2D()(h2_3)
multiply=layers.Multiply()([h2_2,global_ave])
h3=layers.Conv2D(64,(1,1),)(multiply)
concat = layers.Concatenate()([h2_1,h3])
h3=layers.Conv2D(64,(1,1),)(concat)
output=layers.add([input, h3])

model3=keras.Model(inputs=input,outputs=output)
plot_model(model3,to_file='third.png')
Image("third.png")


In [7]:
def myRes_conv(x, s, filters):
  '''
  here the input size changes''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid',name='Conv1_1', kernel_regularizer=lr.l2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # second layer
  x_1 = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='same',name='Conv1_2', kernel_regularizer=lr.l2(0.001))(x)
  x_1 = BatchNormalization()(x_1)
  x_1= Activation(activations.relu)(x_1)
    
  x_2 = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same',name='Conv3_2', kernel_regularizer=lr.l2(0.001))(x)
  x_2 = BatchNormalization()(x_2)
  x_2= Activation(activations.relu)(x_2)

  x_3 = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same',name='Conv3_2_2', kernel_regularizer=lr.l2(0.001))(x)
  x_3 = BatchNormalization()(x_3)
  x_3= Activation(activations.relu)(x_3)
  x_3= global_ave=layers.GlobalAveragePooling2D()(x_3)

  multiply=layers.Multiply()([x_2,x_3])
  
  #third layer
  x3 = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid',name='Conv1_3', kernel_regularizer=lr.l2(0.001))(multiply)
  x3 = BatchNormalization()(x3)
    
  concat = layers.Concatenate()([x_1,x3])

  #Forth layer
  x4 = Conv2D(f2, kernel_size=(3, 3), strides=(1, 1),name='Conv3_4', padding='same', kernel_regularizer=lr.l2(0.001))(concat)
  x4 = BatchNormalization()(x4)
    
  # shortcut 
  x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=lr.l2(0.001))(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x4, x_skip,x3 ])
  x = Activation(activations.relu)(x)

  return x

In [8]:
trainAug = keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1.0 / 255),
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=(-0.05, -0.15), width_factor=(-0.05, -0.15)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.3)
])

In [9]:
img_size=64
inputs = keras.Input(shape=(img_size,img_size,3))
x = trainAug(inputs)
x = myRes_conv(x, s=1, filters=(64, 256))
x = Dense(11, activation='softmax', kernel_initializer='he_normal')(x)
model4 = keras.Model(inputs=inputs, outputs=x, name='Myres')
plot_model(model4,to_file='Myres.png')
Image("Myres.png")


In [10]:
def res_conv(x, s, filters):
  '''
  here the input size changes''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=lr.l2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # second block
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=lr.l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #third block
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=lr.l2(0.001))(x)
  x = BatchNormalization()(x)

  # shortcut 
  x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=lr.l2(0.001))(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

In [11]:
img_size = 64
inputs = keras.Input(shape=(img_size,img_size,3))
x = trainAug(inputs)
x = res_conv(x, s=1, filters=(64, 256))
x = Dense(11, activation='softmax', kernel_initializer='he_normal')(x)
model = keras.Model(inputs=inputs, outputs=x, name='Resnet')
plot_model(model,to_file='Resnet.png')
Image("Resnet.png")

In [12]:
img_size = 64
inputs = keras.Input(shape=(img_size,img_size,3))
x = trainAug(inputs)
x = res_conv(x, s=1, filters=(64, 256))
x = Dense(11, activation='softmax', kernel_initializer='he_normal')(x)
model = keras.Model(inputs=inputs, outputs=x, name='Resnet50')
model.summary()

# Q2:Build your neural networks

In [44]:
img_size = 128
batch_size = 64
train_set = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/foraimednew/train',
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
)
valid_set = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/foraimednew/valid',
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
)
test_set = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/foraimednew/test',
    labels=None,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
)

Model1：我首先嘗試利用Q1題目的第三個Model加入了batch normalization

In [45]:
def myRes_conv(x, s, filters):
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid',name='Conv1_1', kernel_regularizer=lr.l2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # second layer
  x_1 = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='same',name='Conv1_2', kernel_regularizer=lr.l2(0.001))(x)
  x_1 = BatchNormalization()(x_1)
  x_1= Activation(activations.relu)(x_1)
    
  x_2 = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same',name='Conv3_2', kernel_regularizer=lr.l2(0.001))(x)
  x_2 = BatchNormalization()(x_2)
  x_2= Activation(activations.relu)(x_2)

  x_3 = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same',name='Conv3_2_2', kernel_regularizer=lr.l2(0.001))(x)
  x_3 = BatchNormalization()(x_3)
  x_3= Activation(activations.relu)(x_3)
  x_3= global_ave=layers.GlobalAveragePooling2D()(x_3)

  multiply=layers.Multiply()([x_2,x_3])
  
  #third layer
  x3 = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid',name='Conv1_3', kernel_regularizer=lr.l2(0.001))(multiply)
  x3 = BatchNormalization()(x3)
    
  concat = layers.Concatenate()([x_1,x3])

  #Forth layer
  x4 = Conv2D(f2, kernel_size=(3, 3), strides=(1, 1),name='Conv3_4', padding='same', kernel_regularizer=lr.l2(0.001))(concat)
  x4 = BatchNormalization()(x4)
    
  # shortcut 
  x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=lr.l2(0.001))(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x4, x_skip,x3 ])
  x = Activation(activations.relu)(x)

  return x

In [46]:
log_dirs='test-logs/'
model_cbk=keras.callbacks.TensorBoard(log_dir=log_dirs)
model_dirs=log_dirs+'/models'
os.makedirs(model_dirs, exist_ok=True)
save_model=keras.callbacks.ModelCheckpoint(model_dirs+'/MyRes_save.h5',monitor='val_categorical_accuracy',mode='max')

In [47]:
inputs = keras.Input(shape=(img_size,img_size,3))
x = trainAug(inputs)
x = myRes_conv(x, s=1, filters=(64, 256))
x = Flatten()(x)
x = Dense(11, activation='softmax', kernel_initializer='he_normal')(x)
model1 = keras.Model(inputs=inputs, outputs=x, name='MyRes')
model1.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[keras.losses.CategoricalCrossentropy()])
model1.summary()


In [ ]:
model1.fit(train_set, epochs=1, validation_data=valid_set,callbacks=[model_cbk, save_model])

上面演練epoch=1。
但是發現在training時，validation大概在0.2上下震盪，因此就捨棄這個modelQQ

Model2：利用open source撰寫resnet50

In [16]:
def res_identity(x, filters): 
  #renet block where dimension doesnot change.
  #The skip connection is just simple identity conncection
  #we will have 3 blocks and then input will be added

  x_skip = x # this will be used for addition with the residual block 
  f1, f2 = filters

  #first block 
  x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=lr.l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #second block # bottleneck (but size kept same with padding)
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=lr.l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # third block activation used after adding the input
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=lr.l2(0.001))(x)
  x = BatchNormalization()(x)
  # x = Activation(activations.relu)(x)

  # add the input 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x
def res_conv(x, s, filters):
  '''
  here the input size changes''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=lr.l2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # second block
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=lr.l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #third block
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=lr.l2(0.001))(x)
  x = BatchNormalization()(x)

  # shortcut 
  x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=lr.l2(0.001))(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

In [17]:
trainAug = keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1.0 / 255),
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=(-0.05, -0.15), width_factor=(-0.05, -0.15)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.3),    
])

In [18]:
def resnet50():

  inputs = keras.Input(shape=(img_size,img_size,3))
  x = trainAug(inputs)

  # 1st stage
  # here we perform maxpooling, see the figure above

  x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)
  x = MaxPooling2D((3, 3), strides=(2, 2))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

  x = res_conv(x, s=1, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 3rd stage

  x = res_conv(x, s=2, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))

  # 4th stage

  x = res_conv(x, s=2, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))

  # 5th stage

  x = res_conv(x, s=2, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))

  # ends with average pooling and dense connection

  x = AveragePooling2D((2, 2), padding='same')(x)

  x = Flatten()(x)
  x = Dense(11, activation='softmax', kernel_initializer='he_normal')(x) #multi-class

  # define the model 

  model = keras.Model(inputs=inputs, outputs=x, name='Resnet50')

  return model

In [19]:
Model2=resnet50()
Model2.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[keras.losses.CategoricalCrossentropy()])

log_dirs='test-logs/'
model_cbk=keras.callbacks.TensorBoard(log_dir=log_dirs)
model_dirs=log_dirs+'/models'
os.makedirs(model_dirs, exist_ok=True)
save_model=keras.callbacks.ModelCheckpoint(model_dirs+'/Res50_save.h5',monitor='val_categorical_accuracy',mode='max')

In [ ]:
Model2.fit(train_set, epochs=1, validation_data=valid_set,callbacks=[model_cbk, save_model])

此Model跑出來，Kaggle上面最佳為
private public
0.27699 0.28984

model3利用keras的ResNet50，最前面加了augmentation layer

In [21]:
from tensorflow.keras.applications import ResNet50

def build_model(preModel=ResNet50, num_classes=11):
    inputs = keras.Input(shape=(img_size,img_size,3))
    x = trainAug(inputs)
    pred_model = preModel(include_top=False, weights='imagenet',
                          input_shape=(img_size, img_size, 3),
                          pooling='max', classifier_activation='softmax')(x)
    output_layer = layers.Dense(
        num_classes, activation="softmax", name="output_layer")(pred_model)

    model = keras.Model(inputs=inputs,outputs=output_layer)

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

    return model

In [ ]:
model3=build_model()
import os
log_dirs='test-logs'
model_cbk=keras.callbacks.TensorBoard(log_dir=log_dirs)
model_dirs=log_dirs+'/models'
os.makedirs(model_dirs, exist_ok=True)
save_model=keras.callbacks.ModelCheckpoint(model_dirs+'/Resnet50_Aug_save.h5',monitor='val_categorical_accuracy',mode='max')
model3.compile(keras.optimizers.Adam(), 
                loss=keras.losses.CategoricalCrossentropy(from_logits=True), 
                metrics=[keras.metrics.CategoricalAccuracy()])
model3.fit(train_set, epochs=1, validation_data=valid_set,callbacks=[model_cbk, save_model])

此Model跑出來，Kaggle上面最佳為
private public
0.42381 0.41334

Model4:利用課本教的transfer learning。利用keras裡面的ResNet50，並利用imagenet的weights

In [22]:
from tensorflow.keras.applications import ResNet50

def build_model(preModel=ResNet50, num_classes=11):
    inputs = keras.Input(shape=(img_size,img_size,3))
    x = trainAug(inputs)
    pred_model = preModel(include_top=False, weights='imagenet',
                          input_shape=(img_size, img_size, 3),
                          pooling='max', classifier_activation='softmax')(x)
    output_layer = layers.Dense(
        num_classes, activation="softmax", name="output_layer")(pred_model)

    model = keras.Model(inputs=inputs,outputs=output_layer)

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

    return model

In [23]:
Model4=build_model()
import os
log_dirs='test-logs/'
model_cbk=keras.callbacks.TensorBoard(log_dir=log_dirs)
model_dirs=log_dirs+'/models'
os.makedirs(model_dirs, exist_ok=True)
save_model=keras.callbacks.ModelCheckpoint(model_dirs+'/Kear_Resnet50_Aug_save.h5',monitor='val_categorical_accuracy',mode='max')

In [ ]:
Model4.fit(train_set, epochs=1, validation_data=valid_set,callbacks=[model_cbk, save_model])

此Model跑出來，Kaggle上面最佳為
private public
0.53307
0.54382


在與林益帆同學討論後，認為transfer model確實是比較好的方式。但因為redisual block太多並不是太好，因此我們利用課本教的inception v3，並在後面加上residual block來訓練我們的moodel

model5：inceptionv3+one residual block

In [24]:
img_size=299
batch_size=64

In [27]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/foraimednew/train',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        '/kaggle/input/foraimednew/valid',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical')

In [28]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape=(299,299,3),
                                      include_top = False,
                                      weights = 'imagenet')

last_layer = pre_trained_model.get_layer('mixed8') #only use layers including and above mixed8
last_output = last_layer.output  
x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(last_output)
x = BatchNormalization()(x)
x = Activation(activations.relu)(x)


  #2nd stage 
  # frm here on only conv block and identity block, no pooling

x = res_conv(x, s=1, filters=(64, 256))
x = res_identity(x, filters=(64, 256))
x = res_identity(x, filters=(64, 256))

  # ends with average pooling and dense connection

x = AveragePooling2D((2, 2), padding='same')(x)
x = layers.GlobalAveragePooling2D()(x)
x = Dense(11, activation='softmax', kernel_initializer='he_normal')(x) #multi-class  
model5 = tf.keras.Model(pre_trained_model.input,x)
print(model5.summary())

In [ ]:
log_dirs='test-logs/'
model_cbk=keras.callbacks.TensorBoard(log_dir=log_dirs)
model_dirs=log_dirs+'/models'
os.makedirs(model_dirs, exist_ok=True)
starter_learning_rate = 0.01
end_learning_rate = 0.001
decay_steps = 3000
learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(
    starter_learning_rate,
    decay_steps,
    end_learning_rate,
    power=0.5)
save_model=keras.callbacks.ModelCheckpoint(model_dirs+'/Transfer_inceptionv3_save_10.h5',monitor='val_categorical_accuracy',mode='max')
model5.compile(optimizer=tf.keras.optimizers.SGD(
                  learning_rate=learning_rate_fn),
                  loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])



In [ ]:
model5.fit(train_generator, epochs=1, validation_data=validation_generator,callbacks=[model_cbk, save_model])

In [30]:
test_set = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/foraimednew/test',
    labels=None,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    image_size=(img_size, img_size),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
)
test_set = test_set.map(lambda x: (tf.divide(x, 255)))

In [ ]:
pred=model.predict(test_set)
answer=np.zeros(np.shape(pred)[0],dtype=int)
for a in range(np.shape(pred)[0]):
    answer[a]=int(np.argmax(pred[a]))
import pandas as pd
df = pd.DataFrame()
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,np.shape(pred)[0]+1)]
df["Category"] = answer
df.to_csv("inception_10.csv",index = False)

此Model跑出來，Kaggle上面10個epoch最佳為
> private public

> 0.89287 0.90139

此Model跑出來，Kaggle上面20個epoch最佳為
> private public

> 0.90311 0.91035

# Using model to predict test data.

In [32]:
test_set = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/input/foraimednew/test',
    labels=None,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    image_size=(img_size, img_size),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
)
test_set = test_set.map(lambda x: (tf.divide(x, 255)))

In [ ]:
model.load_weights(model_dirs+'/Resnet50_Aug_save.h5')
pred=model.predict(test_set)
answer=np.zeros(np.shape(pred)[0],dtype=int)
for a in range(np.shape(pred)[0]):
    answer[a]=int(np.argmax(pred[a]))
import pandas as pd
df = pd.DataFrame()
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,np.shape(pred)[0]+1)]
df["Category"] = answer
df.to_csv("model3_best.csv",index = False)

# Q3 Data augmentation

本次實作了兩種data augmentation方式。一種是在model最前面加一個augmentation layer、一種是先將data進行augmentation(除了validation and test)再丟到model。詳細方式請看Q2裡面各model作法
> 方法1：https://ithelp.ithome.com.tw/articles/10235805

> 方法2：https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

# Q4. Accuracy on test set and report

本次最好的Model為inception v3的mixed8的output layer接residual block。
我們發現，這一題之所以能有不錯的performance，似乎跟後面接什麼model沒有太大關連。應該是因為Inception v3太過於強大QQ (我單單用InceptionV3 mixed8加上一層convolution並做GlobalAveragePooling2D，epoch=10，accuracy就是0.9，看來後來怎麼架設並沒有影響QQ)

Inception V3是2015年提出的Model。相比於inception V2，v3將convolutional layer分解成不對稱的layer。例如3x3的convolutional layer拆解成1x3 及 3x1的convolutional layer
如此可以減少大量的計算輛。在降低參數量的同時，也增加了模型深度，獲得了不錯的結果

展望：這次因為時間比較趕，所以是直接站在前人的成果上(model&weights)才有不錯的表現，若是按自己建立的model是有點慘不忍賭QQ

若有時間，我會再好好以助教給的第三個model為基礎，好好的建立一個Model，並試著調看看參數來找到這個Model的最佳表現。(但應該可以猜測無法超越inceptionv3的表現)

這次的測驗讓我學會非常多技巧，尤其是對於Resnet有更深的了解。但是對於要選擇哪一個Model或是要建造哪一個model還需要多練習。